# Remove all flow logs

**WARNING**: This script will remove all flow logs from your organization. If the flow logs were created as part of a CloudFormation stack, they will be deleted and there will be drift between the Stack configuration and the Flow Logs. Use this only if you want a fresh start on flow logs throughout an organization.

In [1]:
from jupyterirtools import sso
import os
import time
import importlib
import json

sso.login("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=FBMR-RBJR to activate the session


<IPython.core.display.Javascript object>

In [2]:
import boto3

accounts = []

org_client = boto3.client('organizations')

paginator = org_client.get_paginator('list_accounts')

page_iterator = paginator.paginate()
for page in page_iterator:
    for account in page['Accounts']:
        accounts.append(account['Id'])

ec2_client = boto3.client('ec2')
regions = []

region_response = ec2_client.describe_regions()

for region in region_response['Regions']:
    regions.append(region['RegionName'])



for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in regions:
        session = sso.get_session("Jupyter-IR-AdministratorAccess", account, region_name)
        client = session.client('ec2')
        response = client.describe_flow_logs()

        flow_log_ids = []
        for flow_log in response['FlowLogs']:
            flow_log_id = flow_log['FlowLogId']
            flow_log_ids.append(flow_log_id)

        if len(flow_log_ids) > 0:
            delete_response = client.delete_flow_logs(FlowLogIds=flow_log_ids)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
